# Entrega 2

## 1. Objetivo
+ Este notebook tem como objetivo ajustar as técnicas de estatística para o Business Case Definido (Problema 3 definido abaixo)
+ O objetivo é responder:
    + `O que fizeram os clientes darem Churn?`
    + `Dado um comportamento de compra no mês atual, prever qual será o volume de pedidos no próximo mês.`

+ As bases originais são:
    + `ORDERS - Informações sobre os pedidos realizados.`
    + `MARKETING PUSH FULL - Notificações PUSH ao longo de 6 meses (Junho-Dezembro/2019).`
    + `CUSTOMER SEGMENTATION - Segmentação do cliente.`
    + `ORDERS WITH COST REVENUE - Informações sobre o pedido relacionado, onde verifica-se se o mesmo gerou custo ou receita.`
    + `SESSION VISITS - Comportamento de uso do app.`

    
Criado por Jaime Mishima e Ariel Vicente </br>

**PROBLEMA 1**: Para a pergunta de Churn, optamos por criar um modelo preditivo usando regressão e decison tree.</br>
**PROBLEMA 2**: Para o problema de **importância dos eventos**, optamos por fazer uma análise descritiva. A ser entregue posteriormente.</br></br>

**PROBLEMA 3**: Prever a quantidade de orders por cliente no mês seguinte.</br>

**Ideia:** Criar um modelo preditivo para, através de regressão, prever a quantidade de pedidos de cliente no próximo mês.

**Motivação:** Pelo fato de se ter uma base histórica em mãos, a ideia foi explorar a possibilidade antecipar uma informação valiosa de alguém já conhecido e, assim, ganhar tempo no desenho de estratégias/ações e na tomada de decisão.

**Impacto potencial para o business:**
- Otimizar pushs: Sabendo-se a quantidade de pedidos potenciais para um determinado perfil, o Ifood consegue antecipar a quantidade ideal de pushs no mês.
- Otimizar alocação de entregadores: Sabendo-se a quantidade de pedidos potenciais e onde esses pedidos serão entregues, o Ifood pode trabalhar na otimização do posicionamento dos entregadores (através de incentivos, por exemplo).
- Otimizar parcerias com restaurantes: Sabendo-se a quantidade de pedidos potenciais, onde esses pedidos serão entregues e qual o tipo de comida (que será entregue e a favorita do comprador), o Ifood ganha informações para estimular/alinhar promoções em determinadas regiões e/ou com determinados tipos de restaurante.
- Identificar potenciais Churn/Inativos: Com a previsão da quantidade de pedidos, consequentemente antecipa-se também o ifood_status do cliente.
- Antecipar Marlin tag: Sabendo-se a quantidade de pedidos que serão feitos, o Ifood consegue antecipar a classificação da qualidade do cliente (marlin_tag), podendo criar promoções/ações/pushs específicos para cara um dos tipos de cliente em potencial para fidelizar ainda mais os melhores e estimular (se fizer sentido) a evolução dos demais.

## 2. Imports

In [6]:
from pyspark.sql.functions import udf, count, when, isnull, col, mean, sum, max, avg, min, stddev, count, trim, lower, split, explode
from pyspark.sql.functions import *
from pyspark.sql.functions import collect_list
from pyspark.mllib.stat import Statistics

# tratamento de datas
from pyspark.sql.functions import datediff, to_date, to_timestamp, from_utc_timestamp, round, dayofweek, month

# para o groupby e lag column
import pyspark.sql.functions as f
from pyspark.sql.window import Window
from pyspark.sql import SQLContext
from pyspark.sql.functions import lit

# para a remoção de missing:
from functools import reduce

# para ajuste de type de arrays
from pyspark.sql.types import ArrayType, StringType

# para correlacao
import matplotlib.pyplot as plt
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

# Modeling
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.util import MLUtils

## 3. Help Functions

In [8]:
# Percentual de Missings
from pyspark.sql.functions import count, when, isnull, col
def contar_missing(df):
  """Realiza a contagem da quantidade de missing que existe dentro de um dataframe.
  Args:
    df - Dataframe Spark
  Returns:
    Data Frame spark com apenas 1 linha com a contagem de missing para cada variável.
  """
  aux = []
  for c in df.columns:
    aux.append(count(when(isnull(c), c)).alias(c))
  return df.select(aux)

def percent_missing(df):
  total_linhas = df.count()
  df = contar_missing(df)
  colunas = df.columns
  total_missing = list(df.first().asDict().values())
  valores = zip(colunas, total_missing)
  
  df_aux = spark.createDataFrame(valores, ['variaveis', 'total_missing'])
  df_aux = df_aux.withColumn('perc_missing', col('total_missing') *100/total_linhas)
  return df_aux

# Returns a groupby count of a dataframe (df) column (col)
def percentByCol(df, group_column):
  """Retorna o groupby de uma coluna `col` de um dataframe `df`
  Args:
    df - Dataframe Spark
    group_column - Nome da coluna do dataframe df
  Returns:
    Data Frame spark com o groupby da coluna `col` e uma coluna com o percentual
  """
  return df.groupby(group_column)\
           .count()\
           .withColumnRenamed('count', 'cnt_per_group')\
           .withColumn('percent', f.col('cnt_per_group')*100/f.sum('cnt_per_group').over(Window.partitionBy()))\
           .orderBy('percent', ascending=False)

# Join two dataframes and remove duplicate columns
# Disclaimer: baseado em https://stackoverflow.com/questions/46944493/removing-duplicate-columns-after-a-df-join-in-spark
def join_removing_repeated(df1, df2, cond, how='left'):
    """Retorna o dataframe resultado do join de `df1` e `df2`
    Args:
      df1 - dataframe 1
      df2 - dataframe 2
      cond - chaves para realizar o join
      how - tipo de join (default left)
    Returns:
      Data Frame resultado do join removendo as colunas repetidas
    """
    df = df1.join(df2, cond, how=how)
    repeated_columns = [c for c in df1.columns if c in df2.columns]
    for col in repeated_columns:
        df = df.drop(df2[col])
    return df
  
# Clears the received string x from unwanted characters
def limpeza(x):
  """Retorna a string x após eliminação de caracteres indesejados
    Args:
      x - string
    Returns:
      String tratada
    """
  return x.replace('"', '').replace('\\', '').replace('[', '').replace(']', '')

udf_limpeza = udf(limpeza, StringType()) # create an udf based on limpeza function

# Classifica o número de pushes por ranges
from pyspark.sql.functions import udf
push_range = udf(lambda pushes: '1- < 20' if pushes < 20 else 
                                 '2- 20-60' if (pushes >= 20 and pushes < 60) else
                                 '3- 60-100' if (pushes >= 60 and pushes < 100) else
                                 '4- 100-140' if (pushes >= 100 and pushes < 140) else
                                 '5- 140-180' if (pushes >= 140 and pushes < 180) else
                                 '6- 180-220' if (pushes >= 180 and pushes < 220) else
                                 '7- 220-260' if (pushes >= 220 and pushes < 260) else
                                 '8- 260-300' if (pushes >= 260 and pushes < 300) else
                                 '9- 300+'  if (pushes >= 300) else '')

# Buckets por número de pedidos
from pyspark.sql.functions import udf
order_range = udf(lambda orders: '1- < 5' if orders < 5 else 
                                 '2- 5-10' if (orders >= 5 and orders < 10) else
                                 '3- 0-15' if (orders >= 10 and orders < 15) else
                                 '4- 15-20' if (orders >= 15 and orders < 20) else
                                 '5- 20-25' if (orders >= 20 and orders < 25) else
                                 '6- 25-30' if (orders >= 25 and orders < 30) else
                                 '7- 30-35' if (orders >= 30 and orders < 35) else
                                 '8- 35-40' if (orders >= 35 and orders < 40) else
                                 '9- 40+'  if (orders >= 40) else '')

# Heatmap matriz de correlação
import seaborn as sns
def plot_corr_matrix(correlations,attr,fig_no, figsize=[15,10]):
  """Retorna o heatmap de uma lista de listas com as correlacoes de variaveis
  Args:
    correlations - lista de lista com as correlacoes
    attr - lista com os nomes das variaveis
    fig_no - If not provided, a new figure will be created, and the figure number will be incremented (para o plt.figure)
    figsize 0- tamanho do heatmap
  Returns:
    Heatmap com a matriz de correlacao
    """
  fig=plt.figure(fig_no, figsize=figsize)
  ax=fig.add_subplot(111)
  ax.set_title("Correlacao Variaveis")
  ax = sns.heatmap(correlations, cmap="YlGnBu")
  indice = list(range(1, len(attr)+1))
  indice = [str(s) + ' - ' for s in indice]
  res = [i + j for i, j in zip(indice, attr)] 
  #ax.set_xticks(range(len(filter_colunas_order)))
  ax.set_yticklabels(res)
  plt.yticks(rotation=0) 
  plt.show()
  
# Função para computar o MAPE
def compute_mape(df, y_true='label', y_pred='prediction'):
  mape = df.withColumn('abserror', col(y_true) - col(y_pred))
  mape = mape.withColumn('relerror', abs(col('abserror') / col(y_true)))
  mape = mape.select(round(mean(col('relerror')) * 100, 4).alias('mape'))
  
  return mape.collect()[0][0]

# Mode function to get most frequent values of a dataframe
# Reference: https://stackoverflow.com/questions/45880089/how-to-get-most-frequent-values-of-a-dataframe-in-pyspark
@f.udf
def most_common_udf(x):
    from collections import Counter
    return Counter(x).most_common(1)[0][0]

# Returns feature importances of model
# Reference: https://www.timlrx.com/2018/06/19/feature-selection-using-feature-importance-score-creating-a-pyspark-estimator/
import pandas as pd
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

## 4. Base Treatment
Bases Geradas na entrega 1

### Read Raw Bases

In [11]:
root_dir = '/dbfs/FileStore/ifood'
dbutils.fs.ls(f'{root_dir}')
df_customer_segmentation = spark.read.parquet(f'{root_dir}/customer_segmentation')
df_orders = spark.read.parquet(f'{root_dir}/orders')
df_orders_with_cost_revenue = spark.read.parquet(f'{root_dir}/orders_with_cost_revenue')
df_sessions_visits = spark.read.parquet(f'{root_dir}/sessions_visits')
df_marketing_push_full = spark.read.parquet(f'{root_dir}/marketing_push_full')

### Pushes

In [13]:
# Missing Values Treatment
colunas_pushes_missing = percent_missing(df_marketing_push_full).orderBy('perc_missing', ascending=False)
thresholdMissingPushes = 22
df_columns_to_drop = colunas_pushes_missing.filter(colunas_pushes_missing['perc_missing'] > thresholdMissingPushes).select('variaveis')
list_columns_to_drop = list([row[0] for row in df_columns_to_drop.collect()])
df_mpf = df_marketing_push_full.drop(*list_columns_to_drop)
df_mpf = df_mpf.dropDuplicates()

# Para as colunas que nao remover, separar em categoricas e numericas
df_columns_missing = colunas_pushes_missing.filter(
                                                   (colunas_pushes_missing['perc_missing'] <= thresholdMissingPushes) &
                                                   (colunas_pushes_missing['perc_missing'] > 0)
                                                  ).select('variaveis')
list_df_columns_missing = list([row[0] for row in df_columns_missing.collect()])

filter_colunas_numericas = [x[0] for x in df_mpf[list_df_columns_missing].dtypes if x[1] in ('double', 'int', 'long')]
filter_colunas_categoricas = [x[0] for x in df_mpf[list_df_columns_missing].dtypes if x[1] not in ('double', 'int', 'long')]

# Inputar unknown para colunas categoricas
for coluna in filter_colunas_categoricas:
  df_mpf = df_mpf.fillna('unknown', subset=[coluna])

# Inputar media para colunas numericas
for coluna in filter_colunas_numericas:
  media = df_mpf.agg(mean(coluna)).collect()[0][0]
  df_mpf = df_mpf.fillna(media, subset=[coluna])

# Date Treatment
# importante: transformar primeiro para timestamp, depois para date!!!
df_mpf = df_mpf.withColumn('event_time_utc3', from_utc_timestamp('event_time_utc3', 'UTC'))\
               .withColumn('event_date', from_utc_timestamp('event_date', 'UTC'))\
               .withColumn('event_date', to_date('event_date', 'YYYY-MM-DD'))\
               .withColumn('event_month', month('event_date'))\
               .withColumn('event_dayofweek', dayofweek('event_date'))

### Customer Segmentation

In [15]:
# Remove duplicates
df_customer_segmentation = df_customer_segmentation.distinct()

# Missing Treatment
perc_miss = percent_missing(df_customer_segmentation) # Calcula o percentual de missings para todas as colunas
col_drop = perc_miss.filter((perc_miss['perc_missing'] <= 0.1) & (perc_miss['perc_missing'] > 0)).select('variaveis').rdd.flatMap(lambda x: x).collect() # Seleciona colunas com menos de 0.1% de missings
col_drop.append('customer_id') # Acrescenta a coluna de customer_id na lista de colunas com missing
aux1 = df_customer_segmentation.select(col_drop) # Criar datafram auxiliar apenas com as colunas em col_drop
# Cria lista com o customer_id de quem tem algum dado nulo para alguma das colunas em col_drop:
c_id_drop = aux1.where(reduce(lambda x, y: x | y, (f.col(x).isNull() for x in aux1.columns))).select('customer_id').distinct().rdd.flatMap(lambda x: x).collect() 
# Exclui de df_customer_segmentation todos os registros de quem teve informação nula identificada: (Decisão tomada por conta da base ser histórica e o impacto em número absoluto de resgistros ser pequeno)
df_customer_segmentation = df_customer_segmentation.filter(~df_customer_segmentation.customer_id.isin(c_id_drop))

# Array Treatment
#Ajuste coluna preferred_dishes (tipo array):
df_customer_segmentation = df_customer_segmentation.withColumn('aux_1', udf_limpeza(col('preferred_dishes')))
df_customer_segmentation = df_customer_segmentation.withColumn('preferred_dishes_ar', split(col('aux_1'), ',').cast(ArrayType(StringType())))
#Ajuste coluna top_3_merchants_code (tipo array):
df_customer_segmentation = df_customer_segmentation.withColumn('aux_2', udf_limpeza(col('top_3_merchants_code')))
df_customer_segmentation = df_customer_segmentation.withColumn('top_3_merchants_code_ar', split(col('aux_2'), ',').cast(ArrayType(StringType())))

list_columns_to_drop = ['aux_1','aux_2']
df_customer_segmentation = df_customer_segmentation.drop(*list_columns_to_drop) # Exclusão de colunas auxiliares

# Date Treatment
df_customer_segmentation = df_customer_segmentation.withColumn('registration_date', from_utc_timestamp('registration_date', 'UTC'))\
                                                   .withColumn('registration_date', to_date('registration_date', 'YYYY-MM-DD'))\
                                                   .withColumn('last_valid_order_date', from_utc_timestamp('last_valid_order_date', 'UTC'))\
                                                   .withColumn('last_valid_order_date', to_date('last_valid_order_date', 'YYYY-MM-DD'))\
                                                   .withColumn('last_invalid_order_date', from_utc_timestamp('last_invalid_order_date', 'UTC'))\
                                                   .withColumn('last_invalid_order_date', to_date('last_invalid_order_date', 'YYYY-MM-DD'))\
                                                   .withColumn('first_order_date', from_utc_timestamp('first_order_date', 'UTC'))\
                                                   .withColumn('first_order_date', to_date('first_order_date', 'YYYY-MM-DD'))\
                                                   .withColumn('last_order_date', from_utc_timestamp('last_order_date', 'UTC'))\
                                                   .withColumn('last_order_date', to_date('last_order_date', 'YYYY-MM-DD'))\
                                                   .withColumn('segmentation_month', from_utc_timestamp('segmentation_month', 'UTC'))\
                                                   .withColumn('segmentation_month', to_date('segmentation_month', 'YYYY-MM-DD'))\
                                                   .withColumn('registration_month', month('registration_date'))\
                                                   .withColumn('registration_dayofweek', dayofweek('registration_date'))\
                                                   .withColumn('first_order_month', month('first_order_date'))\
                                                   .withColumn('first_order_dayofweek', dayofweek('first_order_date'))\
                                                   .withColumn('segmentation_month_month', month('segmentation_month'))\
                                                   .withColumn('segmentation_month_dayofweek', dayofweek('segmentation_month'))\
                                                   .withColumn('last_order_month', month('last_order_date'))\
                                                   .withColumn('last_order_dayofweek', dayofweek('last_order_date'))

### Orders and Orders with Cost/Revenue

In [17]:
# Join Orders and Orders with Cost/Revenue bases
df_orders_total = join_removing_repeated(df_orders, df_orders_with_cost_revenue, df_orders.order_number == df_orders_with_cost_revenue.order_number, 'left')

# Missing Treatment
colunas_orders_missing = percent_missing(df_orders_total).orderBy('perc_missing', ascending=False)
# Para as colunas com missing, separar em categoricas e numericas
df_columns_orders_missing = colunas_orders_missing.filter(colunas_orders_missing['perc_missing'] > 0
                                                  ).select('variaveis')
list_df_columns_orders_missing = list([row[0] for row in df_columns_orders_missing.collect()])

filter_colunas_orders_numericas = [x[0] for x in df_orders_total[list_df_columns_orders_missing].dtypes if x[1] in ('double', 'long')]
filter_colunas_orders_numericas_int = [x[0] for x in df_orders_total[list_df_columns_orders_missing].dtypes if x[1] in ('int')]
filter_colunas_orders_categoricas = [x[0] for x in df_orders_total[list_df_columns_orders_missing].dtypes if x[1] not in ('double', 'int', 'long', 'boolean')]
filter_colunas_orders_booleanas = [x[0] for x in df_orders_total[list_df_columns_orders_missing].dtypes if x[1] in ('boolean')]

# Inputar unknown para colunas categoricas
for coluna in filter_colunas_orders_categoricas:
  df_orders_total = df_orders_total.fillna('unknown', subset=[coluna])

# Transformar colunas booleanas para string, depois inputar unknown
for coluna in filter_colunas_orders_booleanas:
  df_orders_total = df_orders_total.withColumn(coluna,col(coluna).cast('string'))
  df_orders_total = df_orders_total.fillna('unknown', subset=[coluna])

# Inputar media para colunas numericas
for coluna in filter_colunas_orders_numericas:
  media = df_orders_total.agg(mean(coluna)).collect()[0][0]
  df_orders_total = df_orders_total.fillna(media, subset=[coluna])
  
# Inputar media para colunas numericas inteiras arredondando para o integer mais perto
for coluna in filter_colunas_orders_numericas:
  media = df_orders_total.agg(mean(coluna)).collect()[0][0]
  df_orders_total = df_orders_total.fillna(media, subset=[coluna])
  df_orders_total = df_orders_total.withColumn(coluna, f.round(df_orders_total[coluna], 0))

# Date Treatment  
# importante: transformar primeiro para timestamp, depois para date!!!
df_orders_total = df_orders_total.withColumn('order_timestamp_local', from_utc_timestamp('order_timestamp_local', 'UTC'))\
                                 .withColumn('last_status_date_local', from_utc_timestamp('last_status_date_local', 'UTC'))\
                                 .withColumn('scheduled_creation_date_local', from_utc_timestamp('scheduled_creation_date_local', 'UTC'))\
                                 .withColumn('order_date_local', from_utc_timestamp('order_date_local', 'UTC'))\
                                 .withColumn('cohort_month', from_utc_timestamp('cohort_month', 'UTC'))\
                                 .withColumn('first_order_date', from_utc_timestamp('first_order_date', 'UTC'))\
                                 .withColumn('order_date_local', to_date('order_date_local', 'YYYY-MM-DD'))\
                                 .withColumn('cohort_month', to_date('cohort_month', 'YYYY-MM-DD'))\
                                 .withColumn('first_order_date', to_date('first_order_date', 'YYYY-MM-DD'))\
                                 .withColumn('order_date_local_month', month('order_date_local'))\
                                 .withColumn('order_date_local_dayofweek', dayofweek('order_date_local'))

### Visits

In [19]:
# Missing Treatment
colunas_visits_missing = percent_missing(df_sessions_visits).orderBy('perc_missing', ascending=False)
# Para as colunas com missing, separar em categoricas e numericas
df_columns_visits_missing = colunas_visits_missing.filter(colunas_visits_missing['perc_missing'] > 0
                                                  ).select('variaveis')
list_df_columns_visits_missing = list([row[0] for row in df_columns_visits_missing.collect()])

filter_colunas_visits_numericas = [x[0] for x in df_sessions_visits[list_df_columns_visits_missing].dtypes if x[1] in ('double', 'long')]
filter_colunas_visits_numericas_int = [x[0] for x in df_sessions_visits[list_df_columns_visits_missing].dtypes if x[1] in ('int')]
filter_colunas_visits_categoricas = [x[0] for x in df_sessions_visits[list_df_columns_visits_missing].dtypes if x[1] not in ('double', 'int', 'long', 'boolean')]
filter_colunas_visits_booleanas = [x[0] for x in df_sessions_visits[list_df_columns_visits_missing].dtypes if x[1] in ('boolean')]

# Inputar unknown para colunas categoricas
for coluna in filter_colunas_visits_categoricas:
  df_sessions_visits = df_sessions_visits.fillna('unknown', subset=[coluna])

# Transformar colunas booleanas para string, depois inputar unknown
for coluna in filter_colunas_visits_booleanas:
  df_sessions_visits = df_sessions_visits.withColumn(coluna,col(coluna).cast('string'))
  df_sessions_visits = df_sessions_visits.fillna('unknown', subset=[coluna])
  
# Inputar media para colunas numericas inteiras arredondando para o integer mais perto
for coluna in filter_colunas_visits_numericas:
  media = df_sessions_visits.agg(mean(coluna)).collect()[0][0]
  df_sessions_visits = df_sessions_visits.fillna(media, subset=[coluna])
  df_sessions_visits = df_sessions_visits.withColumn(coluna, f.round(df_sessions_visits[coluna], 0))
  

# Dates Conversion
df_sessions_visits = df_sessions_visits.withColumn('session_started_at_amsp', from_utc_timestamp('session_started_at_amsp', 'UTC'))\
                                       .withColumn('session_ended_at_amsp', from_utc_timestamp('session_ended_at_amsp', 'UTC'))\
                                       .withColumn('session_started_at_utc0', from_utc_timestamp('session_started_at_utc0', 'UTC'))\
                                       .withColumn('session_ended_at_utc0', from_utc_timestamp('session_ended_at_utc0', 'UTC'))\
                                       .withColumn('session_started_at_utc0', from_utc_timestamp('session_started_at_utc0', 'UTC'))\
                                       .withColumn('session_started_date', to_date('session_started_at_amsp', 'YYYY-MM-DD'))\
                                       .withColumn('session_started_month', month('session_started_date'))\
                                       .withColumn('session_started_dayofweek', dayofweek('session_started_date'))

###Save Tables

In [21]:
%fs rm -r /dbfs/FileStore/treated/

res4: Boolean = true

In [22]:
# List folders
dbutils.fs.ls('/dbfs/FileStore')

# Create folder
treat_dir = '/dbfs/FileStore/treated'
dbutils.fs.mkdirs(f'{treat_dir}')

Out[26]: True

In [23]:
df_mpf.write.parquet('/dbfs/FileStore/treated/df_mpf.parquet')
df_customer_segmentation.write.parquet('/dbfs/FileStore/treated/df_customer_segmentation.parquet')
df_orders_total.write.parquet('/dbfs/FileStore/treated/df_orders_total.parquet')
df_sessions_visits.write.parquet('/dbfs/FileStore/treated/df_sessions_visits.parquet')

In [24]:
# Save as Table
# df_mpf.write.saveAsTable('df_mpf_tratado')
# df_customer_segmentation.write.saveAsTable('df_customer_segmentation_tratado')
# df_orders_total.write.saveAsTable('df_orders_total_tratado')
# df_sessions_visits.write.saveAsTable('df_sessions_visits_tratado')

## 5. ABT Generation

In [26]:
# Read from Parquet
root_dir = '/dbfs/FileStore/treated'
dbutils.fs.ls(f'{root_dir}')
df_customer_segmentation = spark.read.parquet(f'{root_dir}/df_customer_segmentation.parquet')
df_orders_total_tratado = spark.read.parquet(f'{root_dir}/df_orders_total.parquet')
df_sessions_visits_tratado = spark.read.parquet(f'{root_dir}/df_sessions_visits.parquet')
df_mpf_tratado = spark.read.parquet(f'{root_dir}/df_mpf.parquet')

# # Read from saved Tables
# df_customer_segmentation = spark.table("df_customer_segmentation_tratado")
# df_orders_total_tratado = spark.table("df_orders_total_tratado")
# df_sessions_visits_tratado = spark.table("df_sessions_visits_tratado")
# df_mpf_tratado = spark.table("df_mpf_tratado")

In [27]:
# Pushes Grouped by 'external_user_id' and 'event_month'
grouping_cols = ["external_user_id", "event_month"]
other_cols = [c for c in df_mpf_tratado.columns if c not in grouping_cols]
exclude_cols = ['event_channel', 'event_name', 'brand', 'sample_type', 'user_id', 'campaing_id']
other_cols = list(set(other_cols) - set(['event_channel', 'event_name', 'brand', 'sample_type', 'user_id', 'campaing_id']))

# Group Pushes base counting distinct events of a customer in a month
df_mpf_grouped = df_mpf_tratado.filter(df_mpf_tratado['event_name'] == 'received')\
                               .groupBy(grouping_cols).agg(*[countDistinct(c).alias("count_distinct_"+c) for c in other_cols])

# Orders and Sessions join via session_id
df_orders_and_sessions = join_removing_repeated(df_orders_total_tratado, df_sessions_visits_tratado, df_orders_total_tratado.session_id == df_sessions_visits_tratado.session_id, 'left')
df_orders_and_sessions = df_orders_and_sessions.withColumn('order_timestamp_local', from_utc_timestamp('order_timestamp_local', 'UTC'))\
                                               .withColumn('order_timestamp_local', to_date('order_timestamp_local', 'YYYY-MM-DD'))\
                                               .withColumn('order_timestamp_local_month', month('order_timestamp_local'))

# Orders and Sessions grouped by 'customer_id' and 'order_timestamp_local_month'
grouping_cols = ['customer_id','order_timestamp_local_month'] # 'payment_method', 'platform', 'device_model']
other_cols = ['order_total'
              ,'credit'
              ,'paid_amount'
              ,'distance_merchant_customer'
              ,'promo_is_promotion'
              ,'normal_items_quantity'
              ,'promo_items_quantity'
              ,'general_net_profit'
              ,'session_duration_seconds'
              ,'sum_event_open'
              ,'sum_view_restaurant_screen'
              ,'sum_view_dish_screen'
              ,'sum_click_add_item'
              ,'sum_view_checkout'
              ,'sum_callback_purchase'
              ,'order_session_quantity']

# Group Orders/Sessions base summing events of a customer in a month
df_orders_and_sessions_group = df_orders_and_sessions.groupBy(grouping_cols).agg(*[sum(c).alias("sum_"+c) for c in other_cols], count('order_id').alias('number_of_orders'))

# Unifies Pushes, Orders and Sessions user actions in a month
cond = [df_orders_and_sessions_group.customer_id == df_customer_segmentation.customer_id, df_orders_and_sessions_group.order_timestamp_local_month == df_customer_segmentation.segmentation_month_month]
df_join = join_removing_repeated(df_customer_segmentation, df_orders_and_sessions_group, cond, 'left')

cond2 = [df_join.customer_id == df_mpf_grouped.external_user_id,
        df_join.segmentation_month_month == df_mpf_grouped.event_month]
df_final = join_removing_repeated(df_join, df_mpf_grouped, cond2, how='left')

### Save ABT Table

In [29]:
# Save as Table
df_final.write.saveAsTable('df_final_200530')

# Save as parquet
# df_final.write.parquet('/dbfs/FileStore/treated/df_final_200530.parquet')

## 6. O que fizeram os clientes darem Churn?

### 6.1 Pipeline

In [32]:
df_final = spark.table("df_final_200530")

# Cria a Target com 1 para cliente que foi Churn. 0 caso contrário
df_final = df_final.withColumn('target',when(df_final.ifood_status == 'Churn',1).when(df_final.ifood_status == 'Inactive',1).otherwise(0))

# Listas de variaveis por tipo: categorico, numerico e data
categorical_columns = df_final.select(*[x[0] for x in df_final.dtypes if x[1] not in ('double', 'long', 'int', 'bigint', 'date')]).columns
numerical_columns = df_final.select(*[x[0] for x in df_final.dtypes if x[1] in ('double', 'long', 'int', 'bigint')]).columns
date_columns = df_final.select(*[x[0] for x in df_final.dtypes if x[1] in ('date')]).columns

# Exclusão de colunas com dados referentes ao mês. Para melhorar as métricas do modelo, posteriormente iremos criar novas variáveis com regras de diferença entre datas.
excluded_columns = ['ifood_status_last_month'
                    ,'top_3_merchants_code'
                    ,'top_district'
                    ,'top_centroid_id'               
                    ,'registration_month'
                    ,'registration_dayofweek'
                    ,'first_order_month'
                    ,'first_order_dayofweek'
                    ,'segmentation_month_month'
                    ,'segmentation_month_dayofweek'
                    ,'last_order_month'
                    ,'last_order_dayofweek'
                    ,'order_timestamp_local_month' # chave do join da base de order
                    ,'count_distinct_event_dayofweek' # nao tem uma interpretacao. Um 7 diz somente se um usuario recebeu num mes pushes todos os dias da semana
                    ,'event_month' # chave do join da base de pushes
                    ,'event_time_utc3' # timestamp da base de pushes
                    ,'count_distinct_campaign_id' # mesmo significado que count_distinct_campaign_name
                    ,'distance_merchant_customer' # from Orders+Visits: Nulo não faz sentido
                   ]
excluded_columns = list(set(excluded_columns + date_columns + categorical_columns) - set(['marlin_tag', 'last_nps']))
included_columns = list(set(df_final.columns) - set(excluded_columns)) + ['ifood_status','ifood_status_last_month', 'customer_id', 'segmentation_month_month']

df_filtrado = df_final[included_columns]

In [33]:
colunas_missing = percent_missing(df_filtrado)
# Seleciona colunas com missing e Inputa zero
df_columns_missing = colunas_missing.filter((colunas_missing['perc_missing'] > 0)).select('variaveis')

list_df_columns_missing = list([row[0] for row in df_columns_missing.collect()])

filter_colunas_numericas = [x[0] for x in df_filtrado[list_df_columns_missing].dtypes if x[1] in ('double', 'int', 'bigint', 'long')]
filter_colunas_categoricas = [x[0] for x in df_filtrado[list_df_columns_missing].dtypes if x[1] not in ('double', 'int', 'bigint', 'long')]

# Inputar unknown para colunas categoricas
# for coluna in filter_colunas_categoricas:
#   df_mpf = df_mpf.fillna('unknown', subset=[coluna])

# Inputar zero para colunas numericas
for coluna in filter_colunas_numericas:
  df_filtrado = df_filtrado.fillna(0, subset=[coluna])

In [34]:
df_filtrado.count()

Out[11]: 209504

In [35]:
# Based on https://gist.github.com/colbyford/83978917799dbcab6293521a60f29e94
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, OneHotEncoderEstimator, StringIndexer, VectorAssembler

categoricalColumns = ['marlin_tag', 'last_nps']
numericalColumns = list(set(df_filtrado.columns) - set(['customer_id','segmentation_month_month','ifood_status','ifood_status_last_month','target']) - set(categoricalColumns))

categoricalColumnsclassVec = [c + "classVec" for c in categoricalColumns]

stages = []

for categoricalColumn in categoricalColumns:
  print(categoricalColumn)
  # Category Indexing with StringIndexer
  stringIndexer = StringIndexer(inputCol=categoricalColumn, outputCol = categoricalColumn+"Index").setHandleInvalid("skip")
  # Use OneHotEncoder to convert categorical variables into binary SparseVectors
  encoder = OneHotEncoder(inputCol=categoricalColumn+"Index", outputCol=categoricalColumn+"classVec")
  # Add stages.  These are not run here, but will run all at once later on.
  stages += [stringIndexer, encoder]

# Convert label into label indices using the StringIndexer
#label_stringIndexer = StringIndexer(inputCol = label, outputCol = "label").setHandleInvalid("skip")
#stages += [label_stringIndexer]

# Transform all features into a vector using VectorAssembler
assemblerInputs = categoricalColumnsclassVec + numericalColumns 
# assembler only considers 'classVec' columns (it already did not consider stringIndexer)
assembler = VectorAssembler(inputCols = assemblerInputs, outputCol="features")
stages += [assembler]

prepPipeline = Pipeline().setStages(stages)
pipelineModel = prepPipeline.fit(df_filtrado)
dataset = pipelineModel.transform(df_filtrado)

marlin_tag
last_nps

### 6.2. Modelagem: Regressão Logística

In [37]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

treino, teste = dataset.randomSplit([0.8, 0.2], seed = 42)
reglog = LogisticRegression(labelCol='target', )

modeloRL = reglog.fit(treino)
rlPrevisoes = modeloRL.transform(teste)

# Plot de Curva ROC
import matplotlib.pyplot as plt

plt.figure(figsize=(5,5))

plt.plot([0, 1], [0, 1], 'r--')

plt.plot(modeloRL.summary.roc.select('FPR').collect(),
         modeloRL.summary.roc.select('TPR').collect())

plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()

In [38]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score, roc_curve
import seaborn as sns

y_true = rlPrevisoes.select('target')
y_true = y_true.toPandas()

y_pred = rlPrevisoes.select('prediction')
y_pred = y_pred.toPandas()
cm = confusion_matrix(y_true, y_pred)

sns.set(font_scale=1.4)
sns.heatmap(cm, annot=True, annot_kws={"size": 12}, fmt='g')

In [39]:
#print(modelo.coefficients)
#print(modelo.intercept)

print('Acuracia Treino: ', modeloRL.summary.accuracy)
print('Precision Treino: ', modeloRL.summary.precisionByLabel)
print('Recall Treino: ', modeloRL.summary.recallByLabel)
print('areaUnderRoc Treino: ', modeloRL.summary.areaUnderROC)

resultado_teste = modeloRL.evaluate(teste)

print('Acuracia: ', resultado_teste.accuracy)
print('Precision: ', resultado_teste.precisionByLabel)
print('Recall: ', resultado_teste.recallByLabel)
print('areaUnderRoc: ', resultado_teste.areaUnderROC)

Acuracia Treino: 0.9993669752164825
Precision Treino: [0.9991850509582842, 0.9996674531663209]
Recall Treino: [0.9997985360284736, 0.9986553398826191]
areaUnderRoc Treino: 0.9999800523087932
Acuracia: 0.9993341893755647
Precision: [0.9993170694692112, 0.9993629355927884]
Recall: [0.9996204789555581, 0.9988538681948425]
areaUnderRoc: 0.999956713335189

### 6.3. Modelagem: Decision Tree

In [41]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

treinoDT, testeDT = dataset.randomSplit([0.8, 0.2], seed = 42)
reglog = DecisionTreeClassifier(labelCol='target')

modeloDT = reglog.fit(treinoDT)

predicoesDT = modeloDT.transform(testeDT)
resultado = predicoesDT.select("target", "prediction", "probability")

evaluator = BinaryClassificationEvaluator(labelCol='target', metricName='areaUnderROC')
areaUnderROC = evaluator.evaluate(predicoesDT)
print(areaUnderROC)

0.9999943827018923

In [42]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score, roc_curve
import seaborn as sns

y_trueDT = predicoesDT.select('target')
y_trueDT = y_trueDT.toPandas()

y_predDT = predicoesDT.select('prediction')
y_predDT = y_predDT.toPandas()
cm = confusion_matrix(y_trueDT, y_predDT)

sns.set(font_scale=1.4)
sns.heatmap(cm, annot=True, annot_kws={"size": 12}, fmt='g')

### 6.4 Interpretação Resultados

**Resultados:** Tanto na aplicação da Regressão Logística quanto na Árvore de Decisão, obteve-se overfitting. Sabe-se que, apesar dos resultados de acurácia (0.99), os resultados obtidos caracterizam overfitting ao invés de um modelo de alta performance.
  
**Conclusões:** Os resultados obtidos até então não são conclusivos, portanto, ainda não se tem os insumos desejados com a estratégia definida para responder a pergunta do desafio em si. A obtenção de tais insumos será trabalhada para a entrega 3 no processo de melhoria do algoritmo - já destacam-se hipóteses das causas do overfitting (próximos passos).

**Próximos passos:**
- Retirar do modelo variáveis que caracterizam o Churn, no caso, as relacionadas a Orders e Sessões/Visitas - a hipótese é que isso pode estar contribuindo para o overfitting;
- Testar retirar da base os registros de Inativos, que atualmente consideramos como default, e manter apenas os Churns como default - a hipótese é que isso pode estar contribuindo para o overfitting;
- Trabalhar na implementação do cálculo da importância variáveis para que se possa identificar então as que se destacam na justificativa do Churn;
- Incluir resultados descritivos na análise.

## 7. Previsão de quantidade de pedidos por cliente

Objetivo: Prever a quantidade de orders por cliente no mês seguinte.

**Ideia:** Criar um modelo preditivo para, através de regressão, prever a quantidade de pedidos de cliente no próximo mês.

**Motivação:** Pelo fato de se ter uma base histórica em mãos, a ideia foi explorar a possibilidade antecipar uma informação valiosa de alguém já conhecido e, assim, ganhar tempo no desenho de estratégias/ações e na tomada de decisão.

**Impacto potencial para o business:**
- Otimizar pushs: Sabendo-se a quantidade de pedidos potenciais para um determinado perfil, o Ifood consegue antecipar a quantidade ideal de pushs no mês.
- Otimizar alocação de entregadores: Sabendo-se a quantidade de pedidos potenciais e onde esses pedidos serão entregues, o Ifood pode trabalhar na otimização do posicionamento dos entregadores (através de incentivos, por exemplo).
- Otimizar parcerias com restaurantes: Sabendo-se a quantidade de pedidos potenciais, onde esses pedidos serão entregues e qual o tipo de comida (que será entregue e a favorita do comprador), o Ifood ganha informações para estimular/alinhar promoções em determinadas regiões e/ou com determinados tipos de restaurante.
- Identificar potenciais Churn/Inativos: Com a previsão da quantidade de pedidos, consequentemente antecipa-se também o ifood_status do cliente.
- Antecipar Marlin tag: Sabendo-se a quantidade de pedidos que serão feitos, o Ifood consegue antecipar a classificação da qualidade do cliente (marlin_tag), podendo criar promoções/ações/pushs específicos para cara um dos tipos de cliente em potencial para fidelizar ainda mais os melhores e estimular (se fizer sentido) a evolução dos demais.

### 7.1 Pipeline

In [48]:
# df_final = spark.table("df_final_200530")
df_final = spark.table("df_final_200606")


# Cria a Target com 1 para cliente que foi Churn. 0 caso contrário
df_final = df_final.withColumn('target',when(df_final.ifood_status == 'Churn',1).when(df_final.ifood_status == 'Inactive',1).otherwise(0))

# Listas de variaveis por tipo: categorico, numerico e data
categorical_columns = df_final.select(*[x[0] for x in df_final.dtypes if x[1] not in ('double', 'long', 'int', 'bigint', 'date')]).columns
numerical_columns = df_final.select(*[x[0] for x in df_final.dtypes if x[1] in ('double', 'long', 'int', 'bigint')]).columns
date_columns = df_final.select(*[x[0] for x in df_final.dtypes if x[1] in ('date')]).columns

# Exclusão de colunas com dados referentes ao mês. Para melhorar as métricas do modelo, posteriormente iremos criar novas variáveis com regras de diferença entre datas.
excluded_columns = ['ifood_status_last_month'
                    ,'top_3_merchants_code'
                    ,'top_district'
                    ,'top_centroid_id'               
                    ,'registration_month'
                    ,'registration_dayofweek'
                    ,'first_order_month'
                    ,'first_order_dayofweek'
                    ,'segmentation_month_month'
                    ,'segmentation_month_dayofweek'
                    ,'last_order_month'
                    ,'last_order_dayofweek'
                    ,'order_timestamp_local_month' # chave do join da base de order
                    ,'count_distinct_event_dayofweek' # nao tem uma interpretacao. Um 7 diz somente se um usuario recebeu num mes pushes todos os dias da semana
                    #,'sum_promo_is_promotion'
                    ,'event_month' # chave do join da base de pushes
                    ,'event_time_utc3' # timestamp da base de pushes
                    ,'count_distinct_campaign_id' # mesmo significado que count_distinct_campaign_name
                    ,'distance_merchant_customer' # from Orders+Visits: Nulo não faz sentido
                   ]
excluded_columns = list(set(excluded_columns + date_columns + categorical_columns) - set(['marlin_tag', 'last_nps']))
included_columns = list(set(df_final.columns) - set(excluded_columns)) + ['ifood_status','ifood_status_last_month', 'customer_id', 'segmentation_month_month']

df_filtrado = df_final[included_columns]

In [49]:
display(percent_missing(df_filtrado))

variaveis,total_missing,perc_missing
avg_order_total,77610,37.044638765846955
benefits_sensitivity,0,0.0
sum_distance_merchant_customer,77610,37.044638765846955
avg_sum_event_open,81568,38.93386283794104
recency_days,0,0.0
avg_sum_view_checkout,81568,38.93386283794104
avg_sum_view_dish_screen,81568,38.93386283794104
sum_normal_items_quantity,77610,37.044638765846955
number_of_orders,77610,37.044638765846955
target,0,0.0


In [50]:
colunas_missing = percent_missing(df_filtrado)
# Seleciona colunas com missing e Inputa zero
df_columns_missing = colunas_missing.filter((colunas_missing['perc_missing'] > 0)).select('variaveis')

list_df_columns_missing = list([row[0] for row in df_columns_missing.collect()])

filter_colunas_numericas = [x[0] for x in df_filtrado[list_df_columns_missing].dtypes if x[1] in ('double', 'int', 'bigint', 'long')]
filter_colunas_categoricas = [x[0] for x in df_filtrado[list_df_columns_missing].dtypes if x[1] not in ('double', 'int', 'bigint', 'long')]

# Inputar unknown para colunas categoricas
# for coluna in filter_colunas_categoricas:
#   df_mpf = df_mpf.fillna('unknown', subset=[coluna])

# Inputar zero para colunas numericas
for coluna in filter_colunas_numericas:
  df_filtrado = df_filtrado.fillna(0, subset=[coluna])

### 7.1 Pipeline

In [52]:
display(df_filtrado)

benefits_sensitivity,count_distinct_campaign_name,sum_distance_merchant_customer,count_distinct_message_variation_channel,recency_days,number_of_orders,sum_normal_items_quantity,target,recency_days_bucket,days_to_reorder_at_datasource,preferred_shift_bucket,last_nps,count_distinct_platform,orders_last_91d,maturity_orders_bucket,qtt_invalid_orders,freq_last_91d,sum_order_session_quantity,sum_sum_view_restaurant_screen,customer_lifetime_days,was_mub_last_month,sum_sum_view_dish_screen,qtt_orders_last_year,sum_sum_click_add_item,qtt_valid_orders,merchant_offer_bucket,sum_sum_callback_purchase,buyer_last_91d,customer_lifetime_months,marlin_tag,maturity_orders,freq_last_91d_bucket,rfv_score,sum_paid_amount,count_distinct_event_time_utc3,sum_order_total,sum_sum_view_checkout,sum_promo_is_promotion,sum_promo_items_quantity,sum_credit,merchant_variety,count_distinct_event_date,recency_months,sum_session_duration_seconds,avg_aov_last_91d,sum_general_net_profit,sum_sum_event_open,top_dish_bucket,days_to_reorder_at_concluded,merchant_offer,ifood_status,ifood_status_last_month,customer_id,segmentation_month_month
1.0,53,2233.397861257468,2,28.0,1,1.0,0,3.0,32.333333333333336,3,Promoter,1,3.0,4.0,0,0.99,1.0,2.0,282,1,2.0,16,3.0,16,4,1.0,1,9,4. Retention Carp,16.0,1.0,2.0,61.0,54,65.0,7.0,0.0,0.0,4.0,0.6666666666666666,28,0.90322581,4176.0,43.33,10.538,6.0,2,32.333333333333336,457.0,Active,Resurrected,002105cbda8aebd52c1e5bfefb7bad7427c4dbe8616e72351270a4069a6b063e,6
0.0,null,null,null,92.0,null,null,1,5.0,null,3,Sem Avaliacoes,null,1.0,1.0,0,1.0,null,null,92,0,null,1,null,1,5,null,1,3,2. Tilapia,1.0,2.0,3.0,null,null,null,null,null,null,null,1.0,null,3.0,null,74.88,null,null,2,null,2189.0,Inactive,Inactive,003999e7a4460c4c3f0239904d03b16d92c6f7fea7ec699a8311a191a04ef5b5,9
0.75,86,1207.5119034424447,2,5.0,2,6.0,0,1.0,15.25,6,Sem Avaliacoes,1,8.0,5.0,8,2.64,2.0,52.0,2854,1,27.0,36,14.0,192,5,2.0,1,93,2. Tilapia,192.0,3.0,3.0,117.19,99,125.19,9.0,1.0,1.0,8.0,0.875,29,0.16129032,8510.0,83.92,16.700584999999997,29.0,1,15.25,1996.0,Active,Active,00575e37de4cb5073b54cb66763f528a5cbcf796fbc8d5a5dc37ffec9c1b1586,10
0.25,41,13173.528972267713,2,0.0,6,13.0,0,1.0,7.875,6,Promoter,1,16.0,5.0,1,5.27,8.0,20.0,2130,1,42.0,48,15.0,90,5,6.0,1,70,1. Marlin,90.0,4.0,3.0,490.21,53,514.21,28.0,1.0,1.0,24.0,0.625,30,0.0,10500.0,83.87,54.28438463389122,24.0,1,7.875,903.0,Active,Active,013bf1fbda243f14e97bae34dccee95931d856e8421f043d56252738aba69327,8
0.0,38,774.2758670303033,2,18.0,1,2.0,0,3.0,45.5,5,Sem Avaliacoes,1,3.0,2.0,0,0.99,1.0,5.0,109,0,2.0,3,3.0,3,5,1.0,1,3,2. Tilapia,3.0,1.0,2.0,36.0,40,40.0,1.0,0.0,0.0,4.0,1.0,19,0.58064516,2330.0,37.25,7.8755,13.0,3,45.5,892.0,Resurrected,Inactive,0183121fc27c87c3ea002a65cab5c87d765f375c76c4982f2d2444d9c3a632dc,9
0.25,32,3059.777689001008,2,7.0,1,2.0,0,1.0,25.0,6,Promoter,1,4.0,4.0,0,1.32,1.0,2.0,543,0,7.0,12,2.0,15,5,1.0,1,17,2. Tilapia,15.0,2.0,2.0,49.0,46,53.0,5.0,0.0,0.0,4.0,1.0,18,0.22580645,59655.0,61.58,9.7825,7.0,3,25.0,734.0,Resurrected,Inactive,01965817a50b51f72a06f4d707363130a26320079f9b44ce4e0edef8ac9d4766,9
0.3333333333333333,25,2811.1000604785063,2,16.0,1,2.0,0,3.0,30.5,3,Sem Avaliacoes,1,3.0,4.0,1,0.99,1.0,5.0,812,0,2.0,16,2.0,39,4,1.0,1,26,2. Tilapia,39.0,1.0,2.0,44.8,52,48.8,3.0,0.0,0.0,4.0,1.0,18,0.51612903,949.0,42.6,9.2803375,4.0,2,35.666666666666664,155.0,Resurrected,Inactive,01f993ac122e9321d228601e7efcf0f82df1bc64a4d54628ad321dbec610857a,8
0.14285714285714285,62,null,1,88.0,null,null,1,4.0,10.666666666666666,6,Sem Avaliacoes,1,7.0,4.0,2,2.31,null,null,234,0,null,13,null,13,4,null,1,7,1. Marlin,13.0,3.0,3.0,null,62,null,null,null,null,null,1.0,31,2.87096774,null,59.35,null,null,1,13.714285714285715,229.0,Inactive,Inactive,0221bc2d12f754718a157ed800d925aad4427e5cd2e64a92566953d71e26c779,10
0.6666666666666666,3,null,1,48.0,null,null,1,4.0,15.333333333333336,6,Sem Avaliacoes,1,6.0,4.0,1,1.98,null,null,563,0,null,13,null,16,4,null,1,18,2. Tilapia,16.0,2.0,2.0,null,3,null,null,null,null,null,0.666666

In [53]:
df_filtrado_lag2 = df_filtrado.withColumn('number_of_orders_next_month',
                                    f.lag(df_filtrado['number_of_orders'])
                                     .over(Window.partitionBy("customer_id")
                                     .orderBy(desc("segmentation_month_month"))))

df_filtrado_lag2 = df_filtrado_lag2.withColumn('number_of_orders_previous_month',
                                    f.lag(df_filtrado['number_of_orders'])
                                     .over(Window.partitionBy("customer_id")
                                     .orderBy(asc("segmentation_month_month"))))

df_filtrado_lag2 = df_filtrado_lag2.dropna() # usuarios que nao tinham pedidos no mes anterior

# Excluindo variáveis que devem estar altamente correlacionadas com a target:
excl_col = ['target']

incl_col = list(set(df_filtrado_lag2.columns) - set(excl_col))

df_filtrado_lag2 = df_filtrado_lag2[incl_col]

In [54]:
df_filtrado_lag2.count()

Out[6]: 54314

In [55]:
categoricalColumns = ['marlin_tag', 'last_nps','ifood_status','ifood_status_last_month']

numericalColumns = list(set(df_filtrado_lag2.columns) - set(['customer_id','segmentation_month_month','number_of_orders_next_month']) - set(categoricalColumns))

categoricalColumnsclassVec = [c + "classVec" for c in categoricalColumns]

stages = []

for categoricalColumn in categoricalColumns:
  print(categoricalColumn)
  # Category Indexing with StringIndexer
  stringIndexer = StringIndexer(inputCol=categoricalColumn, outputCol = categoricalColumn+"Index").setHandleInvalid("skip")
  # Use OneHotEncoder to convert categorical variables into binary SparseVectors
  encoder = OneHotEncoder(inputCol=categoricalColumn+"Index", outputCol=categoricalColumn+"classVec")
  # Add stages.  These are not run here, but will run all at once later on.
  stages += [stringIndexer, encoder]

# Transform all features into a vector using VectorAssembler
assemblerInputs = categoricalColumnsclassVec + numericalColumns
assembler = VectorAssembler(inputCols = assemblerInputs, outputCol="features")
stages += [assembler]

prepPipeline = Pipeline().setStages(stages)
pipelineModel = prepPipeline.fit(df_filtrado_lag2)
dataset_reg2 = pipelineModel.transform(df_filtrado_lag2)

marlin_tag
last_nps
ifood_status
ifood_status_last_month

### 7.2 Linear Regression

In [57]:
from pyspark.ml.regression import LinearRegression

treino, teste = dataset_reg2.randomSplit([0.8, 0.2], seed=42)

lr = LinearRegression(labelCol='number_of_orders_next_month')

modelo_lr = lr.fit(treino)

print('==============================')
print('Métricas no conjunto de TREINO')
print('==============================')
print(f'Coeficientes: {modelo_lr.coefficients}')
#print(f'pValues: {modelo.summary.pValues}')
print(f'Intercepto: {modelo_lr.intercept}')
print(f'MAE: {modelo_lr.summary.meanAbsoluteError}')
print(f'RMSE: {modelo_lr.summary.rootMeanSquaredError}')
print(f'r2 Ajustado: {modelo_lr.summary.r2adj}')

predicoes = modelo_lr.transform(treino)

print('=============================')
print('Métricas no conjunto de TESTE')
print('=============================')
resultado_teste = modelo_lr.evaluate(teste)

print(f'MAE: {resultado_teste.meanAbsoluteError}')
print(f'RMSE: {resultado_teste.rootMeanSquaredError}')
print(f'r2 Ajustado: {resultado_teste.r2adj}')

predicoes = modelo_lr.transform(teste)
#mape = compute_mape(predicoes, 'number_of_orders')
#print(f'MAPE: {mape}')

==============================
Métricas no conjunto de TREINO
==============================
Coeficientes: [0.045977107571658925,0.07190255370082248,0.03584384257599371,0.020185350461677066,0.08519105497499921,0.07863181174258076,-0.254731568078693,0.5118794008520331,0.1652712887797699,0.3146488485712198,0.029474882420856788,-0.03795955989102729,0.1123541345421825,-0.004895930388032699,-0.0004328072501402788,-1.0523207976135392e-08,0.02292529421594236,-0.0011782885407107477,0.49187785622292396,-0.00039717495995410663,-0.17234764720175297,0.00012781409172222814,0.020320906991759688,0.16030652094563705,0.0471849335036579,-0.011022588828109879,0.011519272474520347,0.0005882243325888292,0.017215164619068225,0.010459428781633227,0.001472861356499157,-3.2030292997629085e-05,-0.033271583368652824,0.0014218655673216062,0.021103527630020075,-0.005541053157565883,0.0007854211129177235,0.007686499962581252,-0.0064684584423242665,-0.05371733272308499,0.00024022531358791732,0.0007854211129177235,-0.0029964924023309185,0.009402719881971573,-5.2075777606901096e-05,-0.0003767603264814102,0.0004202897550944343,-0.0004563985567627756,-0.04731119628000129,0.0010775843713051064,0.0028721230517541737,0.03588878524582454,0.00304984699705948,-0.016481833495868306,-2.8914260728790283e-07,0.0007038997155152396,-0.003561382376248155,-0.0048787757462279285,-0.022536076002897934,-3.077298593727625e-05,4.8537984313083386e-06]
Intercepto: 0.30225599962731153
MAE: 1.3993264452225587
RMSE: 2.3183303768625727
r2 Ajustado: 0.6036170843920943
=============================
Métricas no conjunto de TESTE
=============================
MAE: 1.4053510886971015
RMSE: 2.2887278578958714
r2 Ajustado: 0.6113687332214787

O R quadrado de 0.61 indica que para o modelo de regressão linear, aproximadamente 61% da variabilidade `number_of_orders_next_month` pode ser explicada pelo modelo.

### 7.3 Decision Tree Regressor

In [60]:
# Função auxiliar para exibição de métricas de avaliação:
def reg_model_evaluator(metric):
  return RegressionEvaluator(labelCol="number_of_orders_next_month", predictionCol="prediction", metricName=metric)

In [61]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.util import MLUtils

treino, teste = dataset_reg2.randomSplit([0.8, 0.2], seed=42)

dt = DecisionTreeRegressor(labelCol="number_of_orders_next_month")

modelo = dt.fit(treino)

print('==============================')
print('Métricas no conjunto de TREINO')
print('==============================')

predictions_dt = modelo.transform(treino)

# Exemplos de predições feitas:
predictions_dt.select("prediction", "number_of_orders_next_month", "features").show(5)

mae = reg_model_evaluator('mae').evaluate(predictions_dt)
rmse = reg_model_evaluator('rmse').evaluate(predictions_dt)
r2 = reg_model_evaluator('r2').evaluate(predictions_dt)

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'r2 Ajustado: {r2}')

print(modelo)

print('=============================')
print('Métricas no conjunto de TESTE')
print('=============================')

predictions_dt = modelo.transform(teste)

# Exemplos de predições feitas:
predictions_dt.select("prediction", "number_of_orders_next_month", "features").show(5)

mae = reg_model_evaluator('mae').evaluate(predictions_dt)
rmse = reg_model_evaluator('rmse').evaluate(predictions_dt)
r2 = reg_model_evaluator('r2').evaluate(predictions_dt)

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'r2 Ajustado: {r2}')

print(modelo)

==============================
Métricas no conjunto de TREINO
==============================
+-------------------+---------------------------+--------------------+
 prediction|number_of_orders_next_month| features|
+-------------------+---------------------------+--------------------+
 1.1812750863436945| 0|(61,[1,5,7,11,17,...|
 0.8375305493571352| 0|(61,[0,5,7,11,17,...|
0.31199711780953526| 0|(61,[2,3,7,11,17,...|
0.12552150769673429| 0|(61,[0,5,7,11,17,...|
0.12552150769673429| 0|(61,[0,3,7,10,17,...|
+-------------------+---------------------------+--------------------+
only showing top 5 rows

MAE: 1.4306004140922708
RMSE: 2.3582026357384693
r2 Ajustado: 0.5900743230170624
DecisionTreeRegressionModel (uid=DecisionTreeRegressor_d21b6a9e41b2) of depth 5 with 63 nodes
=============================
Métricas no conjunto de TESTE
=============================
+-------------------+---------------------------+--------------------+
 prediction|number_of_orders_next_month| features|
+-------------------+---------------------------+--------------------+
 0.6522282855843565| 0|(61,[2,3,8,12,17,...|
 0.6522282855843565| 0|(61,[0,5,8,9,17,2...|
0.12552150769673429| 0|(61,[0,3,7,11,17,...|
 1.1812750863436945| 0|[0.0,1.0,0.0,0.0,...|
 2.6352313167259784| 4|[0.0,1.0,0.0,1.0,...|
+-------------------+---------------------------+--------------------+
only showing top 5 rows

MAE: 1.442440309767225
RMSE: 2.35857405970179
r2 Ajustado: 0.5881290871096907
DecisionTreeRegressionModel (uid=DecisionTreeRegressor_d21b6a9e41b2) of depth 5 with 63 nodes

O R quadrado de 0.59 indica que para o modelo de árvore de decisão, aproximadamente 59% da variabilidade `number_of_orders_next_month` pode ser explicada pelo modelo.

Abaixo vemos que as variáveis mais importantes são: `number_of_orders`, `orders_last_91d` e `qq_orders_last_year`. Curioso notar que `number_of_orders_previous_month` não foi uma das principais.

In [64]:
ExtractFeatureImp(modelo.featureImportances, predictions, "features").head(10)

,idx,name,score
5,18,number_of_orders,0.873311
12,25,orders_last_91d,0.061467
21,34,qtt_orders_last_year,0.028344
40,53,recency_months,0.013372
8,21,days_to_reorder_at_datasource,0.007947
46,59,days_to_reorder_at_concluded,0.007309
10,23,number_of_orders_previous_month,0.004231
1,14,count_distinct_campaign_name,0.002163
4,17,recency_days,0.001668
23,36,qtt_valid_orders,0.000189


### 7.4 Random Forest Regressor

In [66]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

treino, teste = dataset_reg2.randomSplit([0.8, 0.2], seed=42)

rf = RandomForestRegressor(labelCol="number_of_orders_next_month")

modelo_rf = rf.fit(treino)

print('==============================')
print('Métricas no conjunto de TREINO')
print('==============================')

predictions_rf = modelo_rf.transform(treino)

# Exemplos de predições feitas:
predictions_rf.select("prediction", "number_of_orders_next_month", "features").show(5)

mae = reg_model_evaluator('mae').evaluate(predictions_rf)
rmse = reg_model_evaluator('rmse').evaluate(predictions_rf)
r2 = reg_model_evaluator('r2').evaluate(predictions_rf)

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'r2 Ajustado: {r2}')

print(modelo_rf)

print('=============================')
print('Métricas no conjunto de TESTE')
print('=============================')

predictions_rf = modelo_rf.transform(teste)

# Exemplos de predições feitas:
predictions_rf.select("prediction", "number_of_orders_next_month", "features").show(5)

mae = reg_model_evaluator('mae').evaluate(predictions_rf)
rmse = reg_model_evaluator('rmse').evaluate(predictions_rf)
r2 = reg_model_evaluator('r2').evaluate(predictions_rf)

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'r2 Ajustado: {r2}')

print(modelo_rf)

==============================
Métricas no conjunto de TREINO
==============================
+-------------------+---------------------------+--------------------+
 prediction|number_of_orders_next_month| features|
+-------------------+---------------------------+--------------------+
 1.0068591586124738| 0|(61,[1,5,7,11,17,...|
 0.6483717209580856| 0|(61,[0,5,7,11,17,...|
0.41501478963507904| 0|(61,[2,3,7,11,17,...|
0.16474219033275805| 0|(61,[0,5,7,11,17,...|
0.13915353851033685| 0|(61,[0,3,7,10,17,...|
+-------------------+---------------------------+--------------------+
only showing top 5 rows

MAE: 1.4132333260281287
RMSE: 2.3246808390399076
r2 Ajustado: 0.6016456594522193
RandomForestRegressionModel (uid=RandomForestRegressor_cf6a6b3d1fdd) with 20 trees
=============================
Métricas no conjunto de TESTE
=============================
+-------------------+---------------------------+--------------------+
 prediction|number_of_orders_next_month| features|
+-------------------+---------------------------+--------------------+
 0.5148365933613475| 0|(61,[2,3,8,12,17,...|
 0.3160140714056577| 0|(61,[0,5,8,9,17,2...|
0.13915353851033685| 0|(61,[0,3,7,11,17,...|
 1.227983787579826| 0|[0.0,1.0,0.0,0.0,...|
 2.5363208533851984| 4|[0.0,1.0,0.0,1.0,...|
+-------------------+---------------------------+--------------------+
only showing top 5 rows

MAE: 1.4235970491093555
RMSE: 2.30888149114838
r2 Ajustado: 0.6053015949432832
RandomForestRegressionModel (uid=RandomForestRegressor_cf6a6b3d1fdd) with 20 trees

O R quadrado de 0.605 indica que para o modelo de Random Forest, aproximadamente 60.5% da variabilidade `number_of_orders_next_month` pode ser explicada pelo modelo.

Abaixo vemos que as variáveis mais importantes são também relacionadas a quantidade de pedidos, como `number_of_orders` e `sum_order_session_quantity`. Interessante notar que ele foi capaz de trazer também um contexto de receita com as variáveis `sum_order_total`, `sum_paid_amount` e `sum_credit`.

In [69]:
ExtractFeatureImp(modelo_rf.featureImportances, predictions_rf, "features").head(10)

,idx,name,score
5,18,number_of_orders,0.413161
16,29,sum_order_session_quantity,0.154827
12,25,orders_last_91d,0.109912
33,46,sum_order_total,0.084282
15,28,freq_last_91d,0.047768
31,44,sum_paid_amount,0.035402
37,50,sum_credit,0.031761
8,21,days_to_reorder_at_datasource,0.030663
46,59,days_to_reorder_at_concluded,0.022337
21,34,qtt_orders_last_year,0.021420


In [70]:
display(predictions_rf.select("prediction", "number_of_orders_next_month", "features"))

prediction,number_of_orders_next_month,features
3.113677660022108,2,"List(1, 50, List(), List(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 7.0, 5179.646100531034, 2.0, 1.0, 4.0, 0.0, 1.0, 12.125, 5.0, 1.0, 7.0, 4.0, 1.0, 2.31, 864.0, 1.0, 21.0, 4.0, 4.0, 1.0, 28.0, 3.0, 2.0, 174.8, 12.0, 12.0, 0.2857142857142857, 7.0, 0.03225806, 57.86, 29.391137500000003, 86.0, 2.0, 13.857142857142858, 161.0))"
2.3192814318524873,4,"List(1, 50, List(), List(0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 30.0, 3622.3265393275537, 2.0, 11.0, 3.0, 0.0, 2.0, 47.5, 5.0, 1.0, 2.0, 4.0, 3.0, 0.66, 764.0, 1.0, 27.0, 4.0, 1.0, 1.0, 25.0, 1.0, 2.0, 75.0, 31.0, 4.0, 1.0, 18.0, 0.35483871, 84.33, 12.891125, 8.0, 3.0, 47.5, 486.0))"
2.278047915244854,4,"List(1, 50, List(), List(0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 36.0, 324.17075183226086, 3.0, 7.0, 3.0, 0.0, 1.0, 110.0, 5.0, 2.0, 1.0, 4.0, 2.0, 0.33, 866.0, 1.0, 19.0, 5.0, 1.0, 1.0, 28.0, 1.0, 1.0, 55.0, 38.0, 4.0, 1.0, 16.0, 0.22580645, 59.0, 9.899, 25.0, 2.0, 124.0, 905.0))"
2.9029935071272694,1,"List(1, 50, List(), List(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.1111111111111111, 48.0, 5964.297048860084, 2.0, 21.0, 3.0, 0.0, 3.0, 11.1, 5.0, 1.0, 9.0, 5.0, 3.0, 2.97, 1120.0, 1.0, 30.0, 5.0, 2.0, 1.0, 36.0, 3.0, 3.0, 139.5, 52.0, 8.0, 0.4444444444444444, 29.0, 0.67741935, 89.99, 22.93975, 19.0, 2.0, 12.333333333333336, 931.0))"
4.2872388025300285,5,"List(1, 50, List(), List(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.13333333333333333, 26.0, 12208.198608426324, 2.0, 6.0, 10.0, 0.0, 1.0, 6.333333333333332, 3.0, 1.0, 15.0, 5.0, 0.0, 4.95, 737.0, 1.0, 35.0, 4.0, 5.0, 1.0, 24.0, 4.0, 3.0, 265.7, 36.0, 20.0, 0.5333333333333333, 25.0, 0.19354839, 45.61, 48.50455, 17.0, 1.0, 6.333333333333332, 273.0))"
3.740570067512594,2,"List(1, 50, List(), List(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.2222222222222222, 21.0, 8726.410474801702, 3.0, 4.0, 19.0, 0.0, 1.0, 12.111111111111109, 5.0, 2.0, 9.0, 4.0, 0.0, 2.97, 767.0, 1.0, 25.0, 5.0, 5.0, 1.0, 25.0, 3.0, 3.0, 418.08000000000004, 59.0, 24.0, 0.5555555555555556, 17.0, 0.12903226, 78.91, 69.41755374999998, 9.0, 1.0, 12.111111111111109, 2353.0))"
2.8625017924404306,4,"List(1, 50, List(), List(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.2857142857142857, 17.0, 4876.401256473897, 2.0, 1.0, 5.0, 0.0, 1.0, 11.5, 5.0, 1.0, 7.0, 4.0, 1.0, 2.31, 927.0, 1.0, 25.0, 3.0, 2.0, 1.0, 30.0, 3.0, 3.0, 93.9, 17.0, 8.0, 0.7142857142857143, 12.0, 0.03225806, 63.43, 15.353506249999999, 13.0, 1.0, 13.142857142857142, 146.0))"
2.551584680993082,3,"List(1, 50, List(), List(0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.2857142857142857, 18.0, 1444.2274752474264, 2.0, 9.0, 1.0, 0.0, 2.0, 14.0, 5.0, 1.0, 7.0, 4.0, 0.0, 2.31, 802.0, 1.0, 19.0, 4.0, 1.0, 1.0, 26.0, 3.0, 3.0, 56.5, 20.0, 4.0, 0.2857142857142857, 13.0, 0.29032258, 63.5, 10.67921875, 8.0, 2.0, 14.0, 174.0))"
2.92475008837084,1,"List(1, 50, List(), List(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.3, 4.0, 6244.582040493615, 2.0, 19.0, 4.0, 0.0, 3.0, 10.5, 6.0, 1.0, 10.0, 4.0, 0.0, 3.3, 824.0, 1.0, 27.0, 5.0, 2.0, 1.0, 27.0, 3.0, 3.0, 179.8, 13.0, 8.0, 0.6, 5.0, 0.61290323, 95.32, 28.82137458960996, 8.0, 1.0, 10.5, 976.0))"
11.711933336575282,16,"List(1, 50, List(), List(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.3142857142857143, 41.0, 22287.413009423643, 2.0, 0.0, 29.0, 0.0, 1.0, 2.771428571428572, 6.0, 1.0, 35.0, 5.0, 4.0, 11.54, 641.0, 1.0, 153.0, 5.0, 23.0, 1.0, 21.0, 5.0, 4.0, 1082.14, 69.0, 60.0, 0.6, 26.0, 0.0, 78.91, 179.31628625, 43.0, 1.0, 2.771428571428572, 1138.0))"


### 7.5 Interpretação Resultados

Abaixo temos para a variável target, `number_of_orders_next_month`, as seguintes estatísticas:
- número de pedidos mínimo: 0 
- número de pedidos máxiomo: 79
- mediana: 1
- média: 2.23

In [73]:
import numpy as np

def median(values_list):
    med = np.median(values_list)
    return float(med)
udf_median = f.udf(median)

display(df_filtrado_lag2.agg(min('number_of_orders_next_month'),udf_median(f.collect_list(col('number_of_orders_next_month'))),max('number_of_orders_next_month'),avg('number_of_orders_next_month')))

min(number_of_orders_next_month),"median(collect_list(number_of_orders_next_month, 0, 0))",max(number_of_orders_next_month),avg(number_of_orders_next_month)
0,1.0,79,2.228750217277942


**Resultados:** Foram rodados três modelos regressores para a predição do número de pedidos do próximo mês (resultados avaliação do modelo na base de teste):
- Regressão Linear: MAE = 1.405; RSME = 2.289; r2 = 0.611;
- Decision Tree Regressor: MAE = 1.442; RSME = 2.359; r2 = 0.588;
- Random Forest Regressor: MAE = 1.4235; RSME = 2.309; r2 = 0.605;

Para a base, a média da target `number_of_orders_next_month` é de 2.2 pedidos, a mediana é de 1. Como o `MAE` para os três modelos é de 1.4 pedido e 50% da base realiza até 1 pedido, o erro ainda é considerável.

O `RMSE` é uma métrica útil pois penaliza erros muito grandes. Como o `RSME` (~2.3) está relativamente próximo do `MAE`, apesar de estarmos errando, a magnitude da distribuição dos erros não indica que temos erros muito elevados.

Com isso, ainda não é possível descartar o uso de um ou outro modelo dado que os resultados foram parecidos e não usou-se técnicas específicas para seleção de variáveis e/ou modelos. MAE e RMSE muito próximos para todos os modelos testados (1.4 e 2.3, respectivamente).
  
**Conclusões:** Até aqui já se pôde validar que a execução da ideia é promissora dado a obtenção de três modelos com r2 próximos a 60% sem muito tempo investido na criação e seleção de variáveis. Porém, ainda é preciso trabalhar alguns pontos específicos (citados nos próximos passos) para então se chegar a resultados consistentes como se espera.

**Próximos passos:**
- Trabalhar mais técnicas relacionadas a séries temporais. Dado que o objetivo é usar a característica temporal da base, espera-se que tais técnicas possam melhorar a performance do modelo.
- Implementar técnicas de seleção de modelos e variáveis após as melhorias supracitadas - como dito anteriormente, ainda não se tem indícios suficientes para descartar o uso de um modelo ou outro.
- Testar o uso da variável `sum_order_total` como target ao invés da variável `number_of_orders_next_month`, isto é, passar a prever o montante gasto pelo cliente ao invés da quantidade de pedidos no próximo mês. Conceitualmente, para o business, os impactos do modelo seriam muito parecidos, contudo, a mudança da previsão de uma variável discreta para uma variável contínua parece uma alternativa interessante (ao menos para um teste).